In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("C:/Users/chels/Downloads/Ronaldo-Dataset.zip")
df.head()

,Unnamed: 0,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,...,lat/lng,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
0,0,10.0,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,...,"45.539131, -122.651648",shot - 30,NaN,20000012,1610612747,10.00,1.0,50.608,54.2000,38.0
1,1,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,...,"45.539131, -122.651648",shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0
2,2,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,...,"45.539131, -122.651648",shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4
3,3,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,...,"45.539131, -122.651648",NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0
4,4,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,...,"45.539131, -122.651648",NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0


In [2]:
pd.set_option("display.max_columns", None)  # Show all columns
print(df.head(10))  # Show first 10 rows

   Unnamed: 0  match_event_id  location_x  location_y  remaining_min  \
0           0            10.0       167.0        72.0           10.0   
1           1            12.0      -157.0         0.0           10.0   
2           2            35.0      -101.0       135.0            7.0   
3           3            43.0       138.0       175.0            6.0   
4           4           155.0         0.0         0.0            NaN   
5           5           244.0      -145.0       -11.0            9.0   
6           6           251.0         0.0         0.0            8.0   
7           7           254.0         1.0        28.0            8.0   
8           8           265.0       -65.0         NaN            6.0   
9           9           294.0       -33.0         NaN            3.0   

   power_of_shot  knockout_match game_season  remaining_sec  distance_of_shot  \
0            1.0             0.0     2000-01           27.0              38.0   
1            1.0             0.0     2000-01 

In [3]:
#USE THIS
# Handling knockout_match (binary categorical)
df["knockout_match"].fillna(method="ffill", inplace=True) #chronological order
# output is just 0 and 1 and it is sorted so can use ffill as it determines forward value to fill

In [4]:
#USE THIS
df["game_season"].fillna(method="ffill", inplace=True) #chronological order

In [5]:
print(df["is_goal"].head(20))

0     NaN
1     0.0
2     1.0
3     0.0
4     1.0
5     0.0
6     1.0
7     NaN
8     1.0
9     0.0
10    0.0
11    1.0
12    1.0
13    0.0
14    0.0
15    0.0
16    NaN
17    1.0
18    0.0
19    NaN
Name: is_goal, dtype: float64


In [6]:
missing_percent = (df["is_goal"].isnull().sum() / len(df)) * 100
print(f"Missing %: {missing_percent:.2f}%")

Missing %: 20.42%


In [7]:
# is goal have 20% missing so dont drop instead used random forest first to train model on basis of factors like distance, range
# shot_baics and stuff then train model and test on missing values more accurate

In [8]:
#USE THIS - IS_GOAL
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Selecting relevant columns
features = ["distance_of_shot", "power_of_shot", "shot_basics", "range_of_shot", "remaining_min"]
df_ml = df[features + ["is_goal"]]

# Encoding categorical variables
df_ml = pd.get_dummies(df_ml, columns=["shot_basics", "range_of_shot"], drop_first=True)

# Splitting data into known and unknown goal outcomes
train_data = df_ml[df_ml["is_goal"].notnull()]
test_data = df_ml[df_ml["is_goal"].isnull()]

# Splitting into X (features) and y (target)
X_train = train_data.drop(columns=["is_goal"])
y_train = train_data["is_goal"]
X_test = test_data.drop(columns=["is_goal"])

# Handling missing values in predictors
imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Training a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicting missing values
df.loc[df["is_goal"].isnull(), "is_goal"] = model.predict(X_test)


In [9]:
print(df.isnull().sum())  # Missing value count for each column

Unnamed: 0                   0
match_event_id            1563
location_x                1461
location_y                1540
remaining_min             1562
power_of_shot             1486
knockout_match               0
game_season                  0
remaining_sec             1594
distance_of_shot          1567
is_goal                      0
area_of_shot              1502
shot_basics               1575
range_of_shot             1564
team_name                 1535
date_of_game              1550
home/away                 1497
shot_id_number            1563
lat/lng                   1565
type_of_shot             15280
type_of_combined_shot    15417
match_id                     0
team_id                      0
remaining_min.1           1535
power_of_shot.1           1539
knockout_match.1          1493
remaining_sec.1           1539
distance_of_shot.1        1568
dtype: int64


In [10]:
print(df["area_of_shot"].value_counts(dropna=False))

Center(C)                12761
Right Side Center(RC)     4562
Right Side(R)             4370
Left Side Center(LC)      3848
Left Side(L)              3573
NaN                       1502
Mid Ground(MG)              81
Name: area_of_shot, dtype: int64


In [11]:
#use this AREA_OF_SHOT
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Select features and target
features = ["shot_basics", "range_of_shot"]
target = "area_of_shot"

# Drop rows where features are missing
df_train = df.dropna(subset=features)

# Encode categorical variables properly
encoder_features = {col: LabelEncoder() for col in features}
for col in features:
    df_train[col] = encoder_features[col].fit_transform(df_train[col].astype(str))

encoder_target = LabelEncoder()
df_train[target] = encoder_target.fit_transform(df_train[target].astype(str))

# Split known values into train & test sets
X_train, X_test, y_train, y_test = train_test_split(df_train[features], df_train[target], test_size=0.2, random_state=42)

# Train a model (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict missing values
df_missing = df[df[target].isna()]
if not df_missing.empty:
    for col in features:
        df_missing[col] = df_missing[col].fillna(df[col].mode()[0])  # Fill missing with mode first
        df_missing[col] = encoder_features[col].transform(df_missing[col].astype(str))  # Encode properly

    df.loc[df[target].isna(), target] = encoder_target.inverse_transform(model.predict(df_missing[features]))

    
#train the model on basis of shot_basics and range   and test on area_of_shot    

C:\Users\chels\AppData\Local\Temp\ipykernel_28348\2476407352.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = encoder_features[col].fit_transform(df_train[col].astype(str))
C:\Users\chels\AppData\Local\Temp\ipykernel_28348\2476407352.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = encoder_features[col].fit_transform(df_train[col].astype(str))
C:\Users\chels\AppData\Local\Temp\ipykernel_28348\2476407352.py:19: SettingWithCopyWarning: 
A value is trying to be set on a 

In [12]:
print(df.isnull().sum())

Unnamed: 0                   0
match_event_id            1563
location_x                1461
location_y                1540
remaining_min             1562
power_of_shot             1486
knockout_match               0
game_season                  0
remaining_sec             1594
distance_of_shot          1567
is_goal                      0
area_of_shot                 0
shot_basics               1575
range_of_shot             1564
team_name                 1535
date_of_game              1550
home/away                 1497
shot_id_number            1563
lat/lng                   1565
type_of_shot             15280
type_of_combined_shot    15417
match_id                     0
team_id                      0
remaining_min.1           1535
power_of_shot.1           1539
knockout_match.1          1493
remaining_sec.1           1539
distance_of_shot.1        1568
dtype: int64


In [13]:
# Step 1: Fill missing DISTANCE_OF_SHOT using RANGE_OF_SHOT median
df["distance_of_shot"] = df.groupby("range_of_shot")["distance_of_shot"].transform(lambda x: x.fillna(x.median()))

# Step 2: Fill remaining missing values with overall median
df["distance_of_shot"] = df["distance_of_shot"].fillna(df["distance_of_shot"].median())



In [14]:
#use this SHOT_BAISCS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Step 1: Encode categorical variables
df_encoded = df.copy()
encoder_range = LabelEncoder()
df_encoded["range_of_shot"] = encoder_range.fit_transform(df_encoded["range_of_shot"])

# Step 2: Prepare training data (drop missing shot_basics)
df_train = df_encoded[df_encoded["shot_basics"].notna()]
df_test = df_encoded[df_encoded["shot_basics"].isna()]

# Encode target variable (shot_basics)
encoder_shot = LabelEncoder()
df_train["shot_basics"] = encoder_shot.fit_transform(df_train["shot_basics"])

# Step 3: Split data
features = ["range_of_shot", "distance_of_shot"]
X_train, X_test, y_train, y_test = train_test_split(df_train[features], df_train["shot_basics"], test_size=0.2, random_state=42)

# Step 4: Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate model accuracy
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Step 6: Predict missing shot_basics
if not df_test.empty:
    df_test["shot_basics"] = model.predict(df_test[features])
    df_test["shot_basics"] = encoder_shot.inverse_transform(df_test["shot_basics"])

    # Fill missing values in original dataframe
    df.loc[df["shot_basics"].isna(), "shot_basics"] = df_test["shot_basics"]


C:\Users\chels\AppData\Local\Temp\ipykernel_28348\2099446472.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["shot_basics"] = encoder_shot.fit_transform(df_train["shot_basics"])


Accuracy: 0.8937339055793991


C:\Users\chels\AppData\Local\Temp\ipykernel_28348\2099446472.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["shot_basics"] = model.predict(df_test[features])
C:\Users\chels\AppData\Local\Temp\ipykernel_28348\2099446472.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["shot_basics"] = encoder_shot.inverse_transform(df_test["shot_basics"])


In [15]:
#use this RANGE_OF_SHOTS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Step 1: Encode categorical features
df_encoded = df.copy()
encoder_shot_basics = LabelEncoder()
df_encoded["shot_basics"] = encoder_shot_basics.fit_transform(df_encoded["shot_basics"])

# Step 2: Prepare training data (drop missing range_of_shot)
df_train = df_encoded[df_encoded["range_of_shot"].notna()]
df_test = df_encoded[df_encoded["range_of_shot"].isna()]

# Encode target variable (range_of_shot)
encoder_range = LabelEncoder()
df_train["range_of_shot"] = encoder_range.fit_transform(df_train["range_of_shot"])

# Step 3: Split into train-test sets
features = ["shot_basics", "distance_of_shot"]
X_train, X_test, y_train, y_test = train_test_split(df_train[features], df_train["range_of_shot"], test_size=0.2, random_state=42)

# Step 4: Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate accuracy
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Step 6: Predict missing range_of_shot
if not df_test.empty:
    df_test["range_of_shot"] = model.predict(df_test[features])
    df_test["range_of_shot"] = encoder_range.inverse_transform(df_test["range_of_shot"])

    # Fill missing values in original dataframe
    df.loc[df["range_of_shot"].isna(), "range_of_shot"] = df_test["range_of_shot"]


C:\Users\chels\AppData\Local\Temp\ipykernel_28348\1454982203.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["range_of_shot"] = encoder_range.fit_transform(df_train["range_of_shot"])


Accuracy: 1.0


C:\Users\chels\AppData\Local\Temp\ipykernel_28348\1454982203.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["range_of_shot"] = model.predict(df_test[features])
C:\Users\chels\AppData\Local\Temp\ipykernel_28348\1454982203.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["range_of_shot"] = encoder_range.inverse_transform(df_test["range_of_shot"])


In [16]:
print(df["date_of_game"].head(20))

0     2000-10-31
1     2000-10-31
2     2000-10-31
3     2000-10-31
4     2000-10-31
5     2000-10-31
6     2000-10-31
7     2000-10-31
8     2000-10-31
9     2000-10-31
10    2000-10-31
11    2000-11-01
12    2000-11-01
13    2000-11-01
14    2000-11-01
15    2000-11-01
16    2000-11-01
17    2000-11-01
18    2000-11-01
19    2000-11-01
Name: date_of_game, dtype: object


In [17]:
print(df["home/away"].head(20))

0       MANU @ POR
1       MANU @ POR
2              NaN
3       MANU @ POR
4       MANU @ POR
5       MANU @ POR
6       MANU @ POR
7       MANU @ POR
8       MANU @ POR
9       MANU @ POR
10             NaN
11    MANU vs. UTA
12    MANU vs. UTA
13    MANU vs. UTA
14    MANU vs. UTA
15    MANU vs. UTA
16    MANU vs. UTA
17    MANU vs. UTA
18    MANU vs. UTA
19    MANU vs. UTA
Name: home/away, dtype: object


In [18]:
#use this DATE OF GAME
# Step 1: Fill using home/away grouping (ffill + bfill)
df["date_of_game"] = df.groupby("home/away")["date_of_game"].transform(lambda x: x.fillna(method="ffill").fillna(method="bfill"))

# Step 2: Fill remaining missing values using global forward & backward fill
df["date_of_game"] = df["date_of_game"].fillna(method="ffill").fillna(method="bfill")

#both depend on each others


In [19]:
#use this home away
# Step 1: Fill missing values using date_of_game grouping
df["home/away"] = df.groupby("date_of_game")["home/away"].transform(lambda x: x.fillna(method="ffill").fillna(method="bfill"))

# Step 2: Fill remaining missing values using global forward & backward fill
df["home/away"] = df["home/away"].fillna(method="ffill").fillna(method="bfill")


In [20]:
print(df.isnull().sum())

Unnamed: 0                   0
match_event_id            1563
location_x                1461
location_y                1540
remaining_min             1562
power_of_shot             1486
knockout_match               0
game_season                  0
remaining_sec             1594
distance_of_shot             0
is_goal                      0
area_of_shot                 0
shot_basics                  0
range_of_shot                0
team_name                 1535
date_of_game                 0
home/away                    0
shot_id_number            1563
lat/lng                   1565
type_of_shot             15280
type_of_combined_shot    15417
match_id                     0
team_id                      0
remaining_min.1           1535
power_of_shot.1           1539
knockout_match.1          1493
remaining_sec.1           1539
distance_of_shot.1        1568
dtype: int64


In [21]:
print(df["shot_id_number"].head(20))

0      1.0
1      2.0
2      3.0
3      4.0
4      5.0
5      6.0
6      7.0
7      8.0
8      9.0
9     10.0
10     NaN
11    12.0
12    13.0
13    14.0
14    15.0
15    16.0
16     NaN
17    18.0
18    19.0
19    20.0
Name: shot_id_number, dtype: float64


In [22]:
#use this SHOT ID NUMBER
df["shot_id_number"] = df["shot_id_number"].interpolate(method="linear")
  #as they are in sequence

In [23]:
#use this LAT/LNG
df["lat/lng"] = df.groupby(["date_of_game", "home/away"])["lat/lng"].ffill().bfill()
#all dependendet on each other

In [75]:
# Define the path to save the file
save_path = r"C:/Users/chels/Downloads/YDS_DATA_CLEANED.csv"

# Save the DataFrame as a CSV file
df.to_csv(save_path, index=False)

print(f"File saved successfully at: {save_path}")


File saved successfully at: C:/Users/chels/Downloads/YDS_DATA_CLEANED.csv
